# Cage building in stk

by: Andrew Tarzia

# Imports 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [7]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock(
    smiles='C1=CC(=C(C=C1Br)CBr)Br',
    functional_groups=[stk.BromoFactory()],
)
cage1 = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix(
        building_blocks=(bb1, bb2),
        optimizer=stk.MCHammer(),
    ),
)

In [8]:
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
cage1 = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix(
        (bb1, bb2),
        vertex_alignments={0: 1},
    ),
)

In [6]:
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
# Produce a Pd+2 atom with 4 functional groups.
palladium_atom = stk.BuildingBlock(
    smiles='[Pd+2]',
    functional_groups=(
        stk.SingleAtom(stk.Pd(0, charge=2))
        for i in range(4)
    ),
    position_matrix=[[0., 0., 0.]],
)

# Build a building block with two functional groups using
# the SmartsFunctionalGroupFactory.
bb1 = stk.BuildingBlock(
    smiles=(
        'C1=NC=CC(C2=CC=CC(C3=C'
        'C=NC=C3)=C2)=C1'
    ),
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ],
)

cage1 = stk.ConstructedMolecule(
    stk.cage.M2L4Lantern(
        building_blocks=(palladium_atom, bb1),
        # Ensure that bonds between the GenericFunctionalGroups
        # of the ligand and the SingleAtom functional groups
        # of the metal are dative.
        reaction_factory=stk.DativeReactionFactory(
            stk.GenericReactionFactory(
                bond_orders={
                    frozenset({
                        stk.GenericFunctionalGroup,
                        stk.SingleAtom
                    }): 9
                }
            )
        ),
#         optimizer=stk.MCHammer(),
    )
)
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [19]:
# Produce a Fe+2 atom with 6 functional groups.
iron_atom = stk.BuildingBlock(
    smiles='[Fe+2]',
    functional_groups=(
        stk.SingleAtom(stk.Fe(0, charge=2))
        for i in range(6)
    ),
    position_matrix=[[0, 0, 0]],
)

# Define coordinating ligand with dummy bromine groups and
# metal coordianting functional groups.
bb2 = stk.BuildingBlock(
    smiles='C1=NC(C=NBr)=CC=C1',
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#35]',
            bonders=(1, ),
            deleters=(),
        ),
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ]
)

# Build iron complex with delta stereochemistry.
iron_oct_delta = stk.ConstructedMolecule(
    stk.metal_complex.OctahedralDelta(
        metals=iron_atom,
        ligands=bb2,
        optimizer=stk.MCHammer(),
    )
)
show_stk_mol(iron_oct_delta)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
# Assign Bromo functional groups to the metal complex.
iron_oct_delta = stk.BuildingBlock.init_from_molecule(
    molecule=iron_oct_delta,
    functional_groups=[stk.BromoFactory()],
)

# Define spacer building block.
bb3 = stk.BuildingBlock(
    smiles=(
        'C1=CC(C2=CC=C(Br)C=C2)=C'
        'C=C1Br'
    ),
    functional_groups=[stk.BromoFactory()],
)

# Build an M4L6 Tetrahedron with a spacer.
cage2 = stk.ConstructedMolecule(
    stk.cage.M4L6TetrahedronSpacer(
        building_blocks=(
            iron_oct_delta,
            bb3,
        ),
        optimizer=stk.MCHammer(),
    )
)

In [21]:
show_stk_mol(cage2)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol